In [1]:
#Import earth engine 
import ee
# Trigger the authentication flow.
ee. Authenticate()
# Initialize the library.
ee. Initialize( project='ee-amanarya1910' )
import folium

# DeltaDTM Elevation Visualization

 This script visualizes elevation data from DeltaDTM using Google Earth Engine and Folium. The code:
 1. Loads DeltaDTM dataset from Earth Engine
 2. Configures visualization parameters for elevation display
 3. Creates an interactive map with base layers and elevation overlay

 Key components:
 - Elevation range: -5m to 20m
 - Red color scheme with 25% opacity
 - 30m resolution
 - Interactive base maps (OpenStreetMap and Google Satellite)
 - Layer controls for toggling views

 The map centers on coordinates [41, -73] with zoom level 5, providing a clear view of elevation patterns in delta regions.

In [11]:
# Load DeltaDTM dataset from Google Earth Engine community catalog
deltaDTM = ee.Image('users/maartenpronk/deltadtm/v1-1')
# deltadtm = ee.Image('users/deltares/deltaDTM/v1_1')

# Mask out no-data values (assuming no-data is represented by negative values)
masked_deltaDTM = deltaDTM.updateMask(deltaDTM.gt(-9999))

# Visualization parameters for elevation data
dtm_vis_params = {
    'min': 0,
    'max': 30,
    'palette': ['#0000ff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000'],
    'opacity': 0.3
}

# Function to add Earth Engine layers to Folium maps
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine method to folium.Map class
folium.Map.add_ee_layer = add_ee_layer

# Center map over New York City
map_center = [40.7128, -74.0060]
dtm_map = folium.Map(location=map_center, zoom_start=9)

# Add DeltaDTM elevation data to the map
dtm_map.add_ee_layer(masked_deltaDTM, dtm_vis_params, 'DeltaDTM Elevation')

# Add basemap (OpenStreetMap)
folium.TileLayer('OpenStreetMap').add_to(dtm_map)

# Layer control and display map
dtm_map.add_child(folium.LayerControl())
display(dtm_map)

 # Interactive Sea Level Rise Projection Analysis

 This notebook implements an interactive sea level rise visualization using DeltaDTM elevation data and IPCC AR6 projections. The code creates a dynamic map that updates based on user-selected years.

 ## Implementation Overview

 The script uses:
 - DeltaDTM elevation data from Earth Engine
 - IPCC AR6 sea level rise projections (SSP2-4.5 scenario)
 - Folium for interactive mapping
 - IPywidgets for year selection

 The core functionality:
 1. Loads elevation and sea level rise data
 2. Calculates potential inundation areas
 3. Creates an interactive map with:
    - Base satellite imagery
    - Elevation visualization
    - Year selection dropdown (2030-2150)
    - Layer controls

 The map updates automatically when a new year is selected, showing potential sea level rise impacts for that time period.

In [6]:
# Load datasets
deltaDTM = ee.Image('users/maartenpronk/deltadtm/v1-1')  # Elevation data
scenario = 'ssp245'  # Medium emissions scenario (SSP2-4.5)

# Create a function to update the map based on selected year
def update_map(year):
    # Get sea level rise projection for the selected year
    ipcc_slr = ee.Image(f'IPCC/AR6/SLP/{scenario}_{year}').select('total_values_quantile_0_5')  # Sea level rise projection
    
    # Compute inundation depth
    # Inundation occurs where elevation < sea level rise; depth is the difference
    difference = ipcc_slr.subtract(deltaDTM)
    inundation_depth = difference.updateMask(difference.gt(0))  # Show only inundated areas (positive depth)
    
    # Visualization parameters
    deltaDTM_vis = {
        'min': 0,           # Elevation in meters
        'max': 100,
        'palette': ['#FF0000'],  # Red for elevation
        'opacity': 0.25,     # 25% opacity as in notebook
        'scale': 30          # 30m resolution
    }
    
    # inundation_vis = {
    #     'min': 0,            # Minimum inundation depth
    #     'max': 2,            # Maximum depth (meters), adjustable based on scenario/year
    #     'palette': ['blue', 'cyan', 'yellow', 'red'],  # Color gradient for depth
    #     'scale': 30          # 30m resolution for high detail
    # }
    
    # Define function to add Earth Engine layers to Folium map
    def addLayer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)
    
    # Attach the addLayer method to Folium's Map class
    folium.Map.addLayer = addLayer
    
    # Create Folium map centered on [41, -73] (near New York, consistent with notebook)
    Map = folium.Map(location=[41, -73], zoom_start=7)
    
    # Add base layers
    basemaps = {
        # 'OpenStreetMap': folium.TileLayer(
        #     tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png',
        #     name='OpenStreetMap',
        #     attr='OpenStreetMap',
        #     overlay=True
        # ),
        'Google Satellite': folium.TileLayer(
            tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            name='Google Satellite',
            attr='Google',
            overlay=True
        )
    }
    # basemaps['OpenStreetMap'].add_to(Map)
    basemaps['Google Satellite'].add_to(Map)
    
    # Add Earth Engine layers
    Map.addLayer(deltaDTM, deltaDTM_vis, 'DeltaDTM Elevation')
    # Map.addLayer(inundation_depth, inundation_vis, f'Inundation Depth ({year})')
    
    # Add layer control for toggling
    Map.add_child(folium.LayerControl())
    
    return Map

# Create widgets for year selection
from ipywidgets import interact, Dropdown

# Create the year selection dropdown
@interact(year=Dropdown(options=['2030', '2040', '2050', '2100', '2150'], value='2100', description='Year:'))
def display_map(year):
    map_display = update_map(year)
    display(map_display)

interactive(children=(Dropdown(description='Year:', index=3, options=('2030', '2040', '2050', '2100', '2150'),…

In [9]:
# # Initialize Earth Engine (assumes authentication is set up)
# ee.Initialize(project='ee-amanarya1910')

# # Load datasets
# # IPCC AR6 SLP dataset for SSP1-2.6, 2150, selecting 50th percentile band
# slp_image = ee.Image('IPCC/AR6/SLP/ssp126_2150').select('total_values_quantile_0_5')

# # Load a DEM (using NASA SRTM 30m for global coverage)
# dem = ee.Image('USGS/SRTMGL1_003')

# # Algorithm: Identify potentially flooded areas
# # Add SLR to DEM and find areas below 0 meters
# future_elevation = dem.add(slp_image)
# flooded_areas = future_elevation.lt(0)  # True (1) where elevation < 0

# # Visualization parameters for flooded areas - only showing flooded regions in blue
# visParams = {
#     'min': 0,
#     'max': 1,
#     'palette': ['#FFFFFF00', '#FF0000'],  # Transparent (not flooded), Red (flooded)
#     'opacity': 0.4
# }

# # Define method to add Earth Engine layers to Folium
# def addLayer(self, ee_image_object, vis_params, name):
#     map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#     folium.raster_layers.TileLayer(
#         tiles=map_id_dict['tile_fetcher'].url_format,
#         attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#         name=name,
#         overlay=True,
#         control=True
#     ).add_to(self)

# # Add the method to Folium's Map class
# folium.Map.addLayer = addLayer

# # Create the map object, centered on a sample location (e.g., near New York)
# Map = folium.Map(location=[41, -73], zoom_start=5)

# # Add basemaps
# basemaps = {
#     'OpenStreetMap': folium.TileLayer(
#         tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png',
#         name='OpenStreetMap',
#         attr='OpenStreetMap',
#         overlay=True
#     ),
#     'Google Satellite': folium.TileLayer(
#         tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
#         name='Google Satellite',
#         attr='Google',
#         overlay=True
#     )
# }
# basemaps['OpenStreetMap'].add_to(Map)
# basemaps['Google Satellite'].add_to(Map)

# # Add the flooded areas layer to the map
# Map.addLayer(flooded_areas, visParams, 'Flooded Areas (SSP1-2.6, 2150)')

# # Add layer control to toggle layers
# Map.add_child(folium.LayerControl())

# # Display the map
# display(Map)


# Sea Level Rise Visualization Script

This script creates an interactive map visualization of sea level rise impacts using Google Earth Engine and Folium. The code:

1. Sets up a custom method to add Earth Engine layers to Folium maps
2. Creates a base map centered on New York (41°N, 73°W)
3. Adds two basemap options:
   - OpenStreetMap for standard street view
   - Google Satellite for aerial imagery
4. Displays flooded areas based on SSP1-2.6 scenario for 2150
5. Includes layer controls to toggle between different map views

The visualization allows users to:
- Switch between different basemaps
- Toggle the flooded areas overlay
- Pan and zoom to explore specific regions
- View the extent of potential flooding under the given scenario

This serves as a foundation for more complex sea level rise analysis and can be extended to include additional scenarios and time periods.


# Sea Level Rise Analysis Script

This script performs a comprehensive analysis of sea level rise impacts using Google Earth Engine (GEE) and the NOAA Sea Level Rise Digital Elevation Model (SLR DEM). Here's a detailed breakdown of the code's functionality:

## Data Loading and Processing
- Loads the NOAA SLR DEM dataset from GEE's public datasets
- Implements a scale detection function to identify image resolutions
- Filters the dataset to specifically use 3-meter resolution data for high precision
- Creates a mosaic of the filtered images and sets the projection to EPSG:3857 (Web Mercator)
- Masks out no-data values (marked as -9999) to ensure data quality

## Terrain Analysis
- Calculates terrain products including:
  - Elevation
  - Slope
  - Aspect
  - Hillshade
  - Roughness
  - Curvature

## Visualization Setup
- Defines two distinct visualization parameter sets:
  1. DEM Visualization:
     - Uses a color gradient from deep blue to red
     - Elevation range: -10m to 50m
     - Helps identify low-lying areas vulnerable to flooding
  
  2. Slope Visualization:
     - Uses a blue color gradient
     - Slope range: 0° to 45°
     - Useful for identifying steep vs. gentle terrain

This script serves as the foundation for sea level rise analysis, providing the necessary data processing and visualization tools for further flood impact assessment.



In [8]:

# Load the NOAA SLR DEM dataset
slrdem = ee.ImageCollection('projects/sat-io/open-datasets/NOAA/SLR_DEM')

# Function to attach nominal scale to images
def add_scale(image):
    b1scale = image.select('b1').projection().nominalScale()
    return image.set('scale', ee.Number(b1scale).round())

# Map the scale function over the collection
ns = slrdem.map(add_scale)

# Filter to get 3m resolution data
ns_3m = ns.filter(ee.Filter.eq('scale', 3))

# Create a mosaic and set projection
dem_image = ns_3m.mosaic().setDefaultProjection('EPSG:3857', None, 3)

# Mask no-data values
masked_dem = dem_image.updateMask(dem_image.gt(-9999))

# Calculate terrain products
terrain = ee.Terrain.products(masked_dem)

# Visualization parameters for DEM
dem_vis_params = {
    'min': -10,
    'max': 50,
    'palette': [
        '#0000ff',  # Deep blue for lowest elevations
        '#00ffff',  # Cyan for low elevations
        '#00ff00',  # Green for medium-low elevations
        '#ffff00',  # Yellow for medium-high elevations
        '#ff0000'   # Red for highest elevations
    ]
}

# Visualization parameters for slope
slope_vis_params = {
    'min': 0,
    'max': 45,
    'palette': ['#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', 
                '#4292c6', '#2171b5', '#08519c', '#08306b']
}

# Define method to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on a coastal area
map_center = [41.0, -73.0]  # New York coastal area
my_map = folium.Map(location=map_center, zoom_start=10)

# Add base layers
basemaps = {
    'OpenStreetMap': folium.TileLayer(
        tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png',
        name='OpenStreetMap',
        attr='OpenStreetMap',
        overlay=True
    ),
    'Google Satellite': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        name='Google Satellite',
        attr='Google',
        overlay=True
    )
}

# Add basemaps to the map
for basemap in basemaps.values():
    basemap.add_to(my_map)

# Add the DEM layer
my_map.add_ee_layer(masked_dem, dem_vis_params, 'NOAA SLR DEM')

# Add the slope layer from terrain products
my_map.add_ee_layer(
    terrain.select('slope'), 
    slope_vis_params, 
    'Slope'
)

# Add a legend for elevation
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white;
            padding: 10px; border-radius: 5px;">
<h4>Elevation (meters)</h4>
<div style="display: flex; flex-direction: column;">
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #ff0000; margin-right: 5px;"></div>
        <div>50m</div>
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #ffff00; margin-right: 5px;"></div>
        <div>25m</div>
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #00ff00; margin-right: 5px;"></div>
        <div>0m (Sea Level)</div>
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #00ffff; margin-right: 5px;"></div>
        <div>-5m</div>
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #0000ff; margin-right: 5px;"></div>
        <div>-10m</div>
    </div>
</div>
</div>
'''

# Add the legend to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Add layer control
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

NOAA SLR DEM Flood Visualization Tool

This script implements a sophisticated coastal flooding visualization system using NOAA's Sea Level Rise Digital Elevation Model (DEM). The tool serves as a critical component for validating flood risk assessments against CoastalDEM's predictions.

## Core Components

### Data Processing Pipeline
- **Source**: NOAA SLR DEM dataset (Google Earth Engine)
- **Resolution**: 3m precision for high-fidelity coastal analysis
- **Data Quality**: Automated masking of invalid values (-9999)

### Key Functions
```python
def add_scale(image):
    # Attaches spatial resolution metadata to each image

def add_ee_layer(self, ee_image_object, vis_params, name):
    # Integrates Earth Engine layers with Folium visualization
```

### Visualization Engine
- **Interactive Map**: Centered on New York's coastal region
- **Flood Simulation**: 2-meter sea level rise scenario
- **Visual Elements**:
  - Flooded areas (red overlay, 50% opacity)
  - Elevation gradient (color-coded)
  - Dynamic layer controls

### Technical Architecture
- **Projection**: EPSG:3857 (Web Mercator)
- **Flood Logic**: Binary classification (elevation ≤ 2m)
- **Integration**: Google Earth Engine + Folium

## Output Specifications
The generated visualization includes:
1. Terrain base layer with elevation coloring
2. Slope analysis overlay
3. Flood risk zones (2m SLR)
4. Interactive elevation legend
5. Layer management controls

This tool provides a foundation for comparative analysis with CoastalDEM's flood risk assessment methodology, enabling validation of inundation predictions across different elevation datasets.

In [ ]:
#Import earth engine 
import ee
# Trigger the authentication flow.
ee. Authenticate()
# Initialize the library.
ee. Initialize( project='ee-amanarya1910' )
import folium

# Load the NOAA SLR DEM dataset
slrdem = ee.ImageCollection('projects/sat-io/open-datasets/NOAA/SLR_DEM')

# Function to attach nominal scale to images
def add_scale(image):
    b1scale = image.select('b1').projection().nominalScale()
    return image.set('scale', ee.Number(b1scale).round())

# Map the scale function over the collection
ns = slrdem.map(add_scale)

# Filter to get 3m resolution data
ns_3m = ns.filter(ee.Filter.eq('scale', 3))

# Create a mosaic and set projection
dem_image = ns_3m.mosaic().setDefaultProjection('EPSG:3857', None, 3)

# Mask no-data values
masked_dem = dem_image.updateMask(dem_image.gt(-9999))

# Create flooded area mask for 2m sea level rise
flooded_mask = masked_dem.lte(2).selfMask()  # Areas where elevation <= 2m

# Visualization parameters for flooded areas
flood_vis_params = {
    'palette': ['#ff0000'],  # Red for flooded areas
    'opacity': 0.5  # Alpha transparency of 0.5
}

# Define method to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on a coastal area
map_center = [41.0, -73.0]  # New York coastal area
# map_center = [25.7617, -80.1918] # Miami 
my_map = folium.Map(location=map_center, zoom_start=10)

# Add base layers
basemaps = {
    'OpenStreetMap': folium.TileLayer(
        tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png',
        name='OpenStreetMap',
        attr='OpenStreetMap',
        overlay=True
    ),
    'Google Satellite': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        name='Google Satellite',
        attr='Google',
        overlay=True
    )
}

# Add basemaps to the map
for basemap in basemaps.values():
    basemap.add_to(my_map)

# Add the flooded areas layer only
my_map.add_ee_layer(flooded_mask, flood_vis_params, 'Flooded Areas (2m SLR)')

# Print a statement before displaying the map
print("Displaying map with 2m sea level rise projection for New York coastal area")

# # Add the legend to the map
# my_map.get_root().html.add_child(folium.Element(legend_html))

# Add layer control
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

Displaying map with 2m sea level rise projection for New York coastal area


In [3]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-amanarya1910')
import folium
